In [ ]:
import os, sys
os.chdir('../')

In [2]:
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import loss as module_loss
import model.metric as module_metric
import model.model as module_arch

import easydict
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

import data_loader.data_loaders as module_data
import model.model as module_arch

from selection.svd_classifier import *
from selection.gmm import *
from selection.util import *

from utils.parse_config import ConfigParser
from utils.util import *
from utils.args import *

In [3]:
config_file = './hyperparams/multistep/config_cifar10_cce_rn34.json'
with open(config_file, 'r') as f:
    config = json.load(f)

# resume_path = './rn34/multistep_asym_40_elr.pth'

In [4]:
def decode(path):
    items = path.split('_')
    noisetype = True if items[1]=='asym' else False
    noiserate = float(items[2]) * 0.01
    
    return noisetype, noiserate, items[3].split('.')[0]

In [5]:
def make_parse(resume_path, config, noise_rate, noisetype):
    parse = easydict.EasyDict({
    "load_name" : resume_path,
    "reinit": False,
    "distill_mode": 'loss'
    })
    
    config['trainer']['percent'] = noise_rate
    config['trainer']['asym'] = noisetype
    
    return parse, config

In [6]:
def extract_cleanidx(teacher, data_loader, parse, print_statistics = True):
    teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    teacher = teacher.cuda()

    if not parse.reinit: teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    for params in teacher.parameters(): params.requires_grad = False
    
    if 'fine' in parse.distill_mode:
        features, labels = get_features(teacher, data_loader)
        clean_labels = fine(current_features=features, current_labels=labels, fit = parse.distill_mode)
    elif 'loss' in parse.distill_mode:
        clean_labels, labels = cleansing_loss(teacher, data_loader)
    else:
        raise NotImplemented 
    if print_statistics: 
        selected, precision, recall, specificity, accuracy = return_statistics(data_loader, clean_labels, datanum=len(labels))
    
    return selected, precision, recall, specificity, accuracy

In [7]:
def make_pd_list(root, config, log_filename):
    random.seed(config['seed'])
    torch.manual_seed(config['seed'])
    torch.cuda.manual_seed_all(config['seed'])
    torch.backends.cudnn.deterministic = True
    np.random.seed(config['seed'])
    
    # load checkpoint path
    pathlist = os.listdir(root)
    pathlist = [path for path in pathlist if ('.pth' in path) and ('eigen' not in path) and ('kmeans' not in path) and ('c100') not in path]
    
    # initialize model
    model = module_arch.resnet34(num_classes=10)
    
    # make pandas file
    logcolumns = ['noisetype', 'noiserate', 'lossfunction', 'selected', 'precision', 'recall', 'specificity', 'accuracy']
    log_pd = pd.DataFrame(np.zeros([len(pathlist), len(logcolumns)]), columns = logcolumns)
    
    # write pandas file
    noisetypelst = ['']
    for i in range(len(pathlist)):
        noisetype, noiserate, lossfunction = decode(pathlist[i])
        parse, config = make_parse('./rn34/' + pathlist[i], config, noiserate, noisetype)
        
        # load original dataloader
        data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size= 100,
        shuffle=False,
        validation_split=0.0,
        num_batches=config['data_loader']['args']['num_batches'],
        training=True,
        num_workers=config['data_loader']['args']['num_workers'],
        pin_memory=config['data_loader']['args']['pin_memory'],
        config=config)
        
        selected, precision, recall, specificity, accuracy = extract_cleanidx(model, data_loader, parse)
        log_pd.loc[i] = [str(noisetype), str(noiserate), lossfunction, selected, precision, recall, specificity, accuracy]
        log_pd.to_csv(log_filename)
        
    return log_pd

In [8]:
make_pd_list(root = './checkpoint/rn34/', config=config, log_filename = 'c10_loss_pretrained_statistics.csv')

Files already downloaded and verified
##############
[3 2 1 1 3 0 0 7 6 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 162.64it/s]


Noisy: 9226, Clean: 40774
Selected samples: 39673 
Precision: 0.9515 
Recall: 0.9258 
Specificity: 0.7915
Accuracy: 0.901 
Fraction of clean samples/selected samples: 0.9515
Files already downloaded and verified


  0%|          | 0/500 [00:00<?, ?it/s]

##############
[8 9 1 9 4 8 3 6 3 6]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


...

100%|██████████| 500/500 [00:03<00:00, 159.72it/s]


Noisy: 7020, Clean: 42980
Selected samples: 29682 
Precision: 0.9801 
Recall: 0.6768 
Specificity: 0.9157
Accuracy: 0.7104 
Fraction of clean samples/selected samples: 0.9801
Files already downloaded and verified
##############
[3 2 1 1 3 2 2 7 6 5]
[3 2 1 1 3 2 2 7 6 5]
Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 152.16it/s]

Noisy: 2458, Clean: 47542
Selected samples: 31404 
Precision: 0.9839 
Recall: 0.6499 
Specificity: 0.7941
Accuracy: 0.657 
Fraction of clean samples/selected samples: 0.9839


,noisetype,noiserate,lossfunction,selected,precision,recall,specificity,accuracy
0,True,0.4,sce,39673.0,0.9515,0.9258,0.7915,0.9010
1,False,0.8,elr,21349.0,0.5865,0.8967,0.7551,0.7946
2,True,0.1,gce,46524.0,0.9956,0.9743,0.9166,0.9714
3,False,0.2,elr,39801.0,0.9949,0.9659,0.9773,0.9680
4,False,0.8,gce,13484.0,0.8730,0.8430,0.9525,0.9219
5,False,0.4,gce,31321.0,0.9807,0.9592,0.9664,0.9618
6,False,0.6,sce,17818.0,0.9778,0.7589,0.9854,0.8814
7,False,0.2,sce,28369.0,0.9898,0.6850,0.9678,0.7359
8,True,0.3,sce,24739.0,0.9340,0.5376,0.7675,0.5699
9,False,0.6,cce,15661.0,0.9812,0.6693,0.9891,0.8423
